In [ ]:
# Cell 1: Install dependencies
!pip install -q torch transformers peft accelerate

# Cell 2: Load your fine-tuned model (no training needed)
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
LORA_MODEL = "shubhampareek/interview-coach-lora"  # Your HuggingFace model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(LORA_MODEL)
tokenizer.pad_token = tokenizer.eos_token

# Load model with LoRA
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, LORA_MODEL)
model.eval()

print("✅ Model loaded!")


In [ ]:
# Define resume and prompt first
test_resume = """
MICHAEL RODRIGUEZ
DevOps Engineer & Cloud Architect

michael.rodriguez@email.com
Phone: (555) 456-7890
Location: Austin, TX

-----------------------------------------------------------
CERTIFICATIONS
-----------------------------------------------------------
- AWS Solutions Architect Professional (2023)
- Certified Kubernetes Administrator (CKA) (2022)
- HashiCorp Terraform Associate (2022)
- AWS DevOps Engineer Professional (2021)

-----------------------------------------------------------
TECHNICAL SKILLS
-----------------------------------------------------------
Cloud Platforms:     AWS, GCP, Azure
Containerization:    Docker, Kubernetes, ECS, EKS
IaC:                 Terraform, CloudFormation, Pulumi
CI/CD:               Jenkins, GitLab CI, GitHub Actions, ArgoCD
Monitoring:          Prometheus, Grafana, DataDog, ELK Stack
Scripting:           Python, Bash, Go
Version Control:     Git, GitHub, GitLab

-----------------------------------------------------------
WORK EXPERIENCE
-----------------------------------------------------------
Lead DevOps Engineer
Amazon Web Services | 2021 - Present

- Architected multi-region infrastructure supporting 99.99% uptime
- Implemented GitOps workflow reducing deployment failures by 70%
- Managed infrastructure costs, achieving $2M annual savings
- Led migration of 200+ microservices to Kubernetes

DevOps Engineer
Stripe | 2018 - 2021

- Built automated testing pipeline with 90% code coverage
- Designed disaster recovery system with 15-minute RTO
- Created self-service deployment platform for 50+ developers

-----------------------------------------------------------
EDUCATION
-----------------------------------------------------------
B.S. Computer Engineering
Texas A&M University, 2018
```
"""

prompt = f"""Here is the candidate's resume:

{test_resume}

Generate 5 specific interview questions based on this resume."""

# Now generate
response = generate_questions(prompt)
print(response)

In [ ]:
# More creative/varied questions
response = generate_questions(prompt, max_tokens=500)

# Or modify the function call with different temperature
def generate_questions_custom(prompt, temperature=0.9, max_tokens=400):
    messages = [
        {"role": "system", "content": "You are an expert interviewer. Generate specific, insightful interview questions."},
        {"role": "user", "content": prompt}
    ]

    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,  # Higher = more creative
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("[/INST]")[-1].strip()

# Test
response = generate_questions_custom(prompt, temperature=0.9)
print(response)

In [ ]:
# Cell 4: Upload and process exported prompts
from google.colab import files
import json

uploaded = files.upload()  # Upload exported_prompts.json

with open("exported_prompts.json", "r") as f:
    data = json.load(f)

# Cell 5: Generate questions
results = []
for item in data["prompts"][:3]:  # Test with first 3
    print(f"Generating: {item['id']}")
    response = generate_questions(item["prompt"])
    results.append({"id": item["id"], "questions": response})
    print(response[:300] + "...\n")

print("✅ Done!")